In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet152
from tqdm import tqdm
import time
import copy
import os
import csv
from torch.utils.data import Dataset
from PIL import Image

In [2]:
# Channel Attention Module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

# Spatial Attention Module
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

# CBAM Module
class CBAM(nn.Module):
    def __init__(self, planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(planes, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.channel_attention(x)
        out = out * self.spatial_attention(out)
        return out

In [3]:
class ResNet152_CBAM(nn.Module):
    def __init__(self, pretrained=True, num_classes=2):
        super(ResNet152_CBAM, self).__init__()
        self.model = resnet152(pretrained=pretrained)

        # CBAM modules
        self.cbam1 = CBAM(256)   # After layer1
        self.cbam2 = CBAM(512)   # After layer2
        self.cbam3 = CBAM(1024)  # After layer3
        self.cbam4 = CBAM(2048)  # After layer4

        # Modify the classifier
        self.model.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)

        x = self.model.layer1(x)
        x = self.cbam1(x)

        x = self.model.layer2(x)
        x = self.cbam2(x)

        x = self.model.layer3(x)
        x = self.cbam3(x)

        x = self.model.layer4(x)
        x = self.cbam4(x)

        x = self.model.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.model.fc(x)

        return x

In [4]:
data_dir = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data'
batch_size = 16
num_workers = 4
num_classes = 100

# Data augmentations and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# Datasets and Dataloaders
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}",
                                          transform=data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=num_workers)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            loop = tqdm(dataloaders[phase], total=len(dataloaders[phase]),
                        desc=f"{phase.capitalize()} Epoch {epoch+1}")

            for inputs, labels in loop:
                inputs = inputs.to(device)
                labels = labels.to(device)


                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                loop.set_postfix(loss=running_loss /
                                 ((loop.n + 1) * batch_size),
                                 acc=running_corrects.double().item() /
                                 ((loop.n + 1) * batch_size))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        scheduler.step()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m\
          {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model


In [6]:
# Initialize CBAM-ResNet152
model_cbam = ResNet152_CBAM(pretrained=True, num_classes=num_classes)
model_cbam = model_cbam.to(device)

# Loss function, implement label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Optimizer (finetune all parameters)
optimizer = optim.Adam(model_cbam.parameters(), lr=0.0001)

# Learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


/home/gpl/anaconda3/envs/VisualRecog/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gpl/anaconda3/envs/VisualRecog/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
num_epochs = 100
model_cbam = train_model(model_cbam, criterion, optimizer,
                         exp_lr_scheduler, num_epochs=num_epochs)

# Save the best model
torch.save(model_cbam.state_dict(),
           'resnet152_cbam_labelsmoothing_best100.pth')
print('Model saved as resnet152_cbam_labelsmoothing_best100.pth')

Epoch 1/100
----------


Train Epoch 1: 100%|██████████| 1296/1296 [12:21<00:00,  1.75it/s, acc=0.526, loss=2.46]


train Loss: 2.4645 Acc: 0.5263


Val Epoch 1: 100%|██████████| 19/19 [00:17<00:00,  1.06it/s, acc=0.602, loss=2.05]


val Loss: 2.0732 Acc: 0.6100
Epoch 2/100
----------


...

val Loss: 1.7522 Acc: 0.8133
Epoch 100/100
----------


Train Epoch 100: 100%|██████████| 1296/1296 [07:17<00:00,  2.96it/s, acc=0.922, loss=1.09]


train Loss: 1.0909 Acc: 0.9225


Val Epoch 100: 100%|██████████| 19/19 [00:02<00:00,  8.27it/s, acc=0.799, loss=1.52]


val Loss: 1.5431 Acc: 0.8100
Training complete in 711m 34s
Best val Acc: 0.8533
Model saved as resnet152_cbam_labelsmoothing_best100.pth


In [10]:
# Load the trained model with CBAM
model = ResNet152_CBAM(num_classes=len(class_names))
model.load_state_dict(torch.load('resnet152_cbam_labelsmoothing_best100.pth'))
model = model.to(device)
model.eval()

# Define the transform (same as validation)
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

val_folder = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data/val/'
val_dataset = datasets.ImageFolder(val_folder, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,
                                         shuffle=False)

correct = 0
total = 0
all_predictions = []

with torch.no_grad():
    for i, (inputs, labels) in enumerate(tqdm(val_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        true_label = labels.item()
        predicted_label = preds.item()

        correct += (predicted_label == true_label)
        total += 1

        # Get the file path of the current sample
        img_path, _ = val_dataset.samples[i]
        all_predictions.append((img_path.split('.')[0], predicted_label))

# Save predictions to a CSV
fname = 'val_predictions_resnet152_w_CBAM_labelsmoothing.csv'
with open(fname, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'pred_label'])  # header
    writer.writerows(all_predictions)


accuracy = correct / total * 100
print(f'Validation Accuracy: {accuracy:.2f}%')

100%|██████████| 300/300 [00:07<00:00, 41.26it/s]

Validation Accuracy: 85.33%


In [11]:
# Load the Model
model = ResNet152_CBAM(num_classes=len(class_names))
model_path = 'resnet152_cbam_labelsmoothing_best100.pth'
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

# Transforms (same as val transforms)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset and Dataloader for test set
test_folder = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data/test'

class TestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, img_name

# Create the dataset and loader
test_dataset = TestDataset(test_folder, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=1, shuffle=False)

# Class names (same order as training classes)
class_names = image_datasets['train'].classes

# Run Inference on the Test Set
predictions = []

with torch.no_grad():
    for inputs, img_names in tqdm(test_loader, desc="Testing"):
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        predicted_label = preds.item()
        predicted_class = class_names[predicted_label]

        # Save predictions
        predictions.append((os.path.basename(img_names[0]).split('.')[0],
                            predicted_class))

predictions.sort(key=lambda x: x[0])

# Save predictions to a CSV
fname = 'prediction_resnet152_w_CBAM_labelsmoothing.csv'
with open(fname, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'pred_label'])  # header
    writer.writerows(predictions)

print("Predictions for test set saved to\
      prediction_resnet152_w_CBAM_labelsmoothing.csv")


Testing: 100%|██████████| 2344/2344 [00:57<00:00, 41.01it/s]

Predictions for test set saved to prediction_resnet152_w_CBAM_labelsmoothing.csv


In [12]:
# Load the Model
model_path = 'resnet152_cbam_labelsmoothing_best100.pth'
model.load_state_dict(torch.load(model_path))

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())

# Number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters()
                       if p.requires_grad)

print(f'Total parameters: {total_params}')
print(f'Trainable parameters: {trainable_params}')

Total parameters: 59045420
Trainable parameters: 59045420
